In [2]:
import xarray as xr
import pandas as pd
import os

In [4]:
print(os.getcwd())
#Load the dataset
times_ds = xr.open_dataset('sg194_May_2025_UW_OTTER_(SG194)_timeseries.nc')
display(times_ds)

#Align time with sg_data_point and apply offset (if needed)
adjusted_time = pd.to_datetime(times_ds['time'].values) + pd.DateOffset(years=0, months=0, days=0)

times_ds['PAR_470nm'] = times_ds['eng_wlbb2fl_sig470nm']
times_ds['particle_concentration_700nm'] = times_ds['eng_wlbb2fl_sig700nm']
times_ds['chlorophyll_695nm'] = times_ds['eng_wlbb2fl_sig695nm']

times_ds = times_ds.assign_coords(time=('sg_data_point', adjusted_time))

#Select the relevant variables
new_times_ds = times_ds[['time', 'depth', 'latitude', 'longitude','temperature', 'salinity', 'dissolved_oxygen_sat', 'PAR_470nm', 'particle_concentration_700nm', 'chlorophyll_695nm']]

#Convert to DataFrame and save
new_times_ds.to_dataframe().reset_index().to_csv('sg194_May_2025_UW_OTTER_(SG194)_timeseries_cleaned.csv', index=False)
new_times_ds.to_netcdf('sg194_May_2025_UW_OTTER_(SG194)_timeseries_cleaned.nc')


C:\Users\lydia\Seagliders


<xarray.Dataset> Size: 19MB
Dimensions:                                   (gps_info: 1167,
                                               sg_data_point: 105367,
                                               trajectory: 389, dive: 389)
Coordinates:
    ctd_time                                  (sg_data_point) datetime64[ns] 843kB ...
    ctd_depth                                 (sg_data_point) float32 421kB ...
    latitude                                  (sg_data_point) float32 421kB ...
    longitude                                 (sg_data_point) float32 421kB ...
  * trajectory                                (trajectory) int32 2kB 1 2 ... 389
Dimensions without coordinates: gps_info, sg_data_point, dive
Data variables: (12/66)
    gps_info_dive_number                      (gps_info) int32 5kB ...
    sg_data_point_dive_number                 (sg_data_point) int32 421kB ...
    log_gps_time                              (gps_info) datetime64[ns] 9kB ...
    time                                      (sg_data_point) datetime64[ns] 843kB ...
    pressure                                  (sg_data_point) float32 421kB ...
    depth                                     (sg_data_point) float32 421kB ...
    ...                                        ...
    end_longitude                             (dive) float32 2kB ...
    depth_avg_curr_east                       (dive) float32 2kB ...
    depth_avg_curr_north                      (dive) float32 2kB ...
    depth_avg_curr_qc                         (dive) |S1 389B ...
    latlong_qc                                (dive) |S1 389B ...
    glider                                    |S12 12B ...
Attributes: (12/47)
    project:                         May 2025 UW OTTER (SG194)
    title:                           Physical, chemical, and biological data ...
    summary:                         SG194 May 2025 UW OTTER (SG194)
    source:                          Seaglider SG194
    references:                      http://data.nodc.noaa.gov/accession/0092291
    processing_level:                1.12
    ...                              ...
    date_modified:                   2025-06-13T17:06:17Z
    uuid:                            e9774cfa-487c-11f0-80b3-07c04838153a
    base_station_version:            3.0.2
    base_station_micro_version:      0
    quality_control_version:         1.12
    Conventions:                     CF-1.6